In [19]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np
import glob
import os
import re

In [13]:
def read_img(filename):
        dataset = gdal.Open(filename)
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        im_geotrans = dataset.GetGeoTransform() #仿射矩阵
        im_proj = dataset.GetProjection() #地图投影
        im_data = dataset.ReadAsArray(0,0,im_width,im_height)
        del dataset
        return im_proj,im_geotrans,im_data

def write_img(filename,im_proj,im_geotrans,im_data):
        if "int8" in im_data.dtype.name:
            datatype = gdal.GDT_Byte
        if "int16" in im_data.dtype.name:
            datatype = gdal.GDT_UInt16
        else:
            datatype = gdal.GDT_Float32

        if len(im_data.shape) == 3:
            im_bands, im_height, im_width = im_data.shape
        else:
            im_bands, (im_height, im_width) = 1, im_data.shape

        driver = gdal.GetDriverByName("GTiff")  # 数据类型必须有，因为要计算需要多大内存空间
        dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)

        dataset.SetGeoTransform(im_geotrans)  # 写入仿射变换参数
        dataset.SetProjection(im_proj)  # 写入投影

        if im_bands == 1:
            dataset.GetRasterBand(1).WriteArray(im_data)  # 写入数组数据
        else:
            for i in range(im_bands):
                dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
        del dataset

In [14]:
proj = read_img("F:\\SCI\\CESM\\CESM2.hist.AODT.2007-05.tif")[0]
geotrans = read_img("F:\\SCI\\CESM\\CESM2.hist.AODT.2007-05.tif")[1]
proj

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [15]:
y_array = read_img("F:\\SCI\\MOD\\DBAOD550\\tif\\MYD.DB.AOD550.200301.tif")[2]
write_img("test.tif",proj,geotrans,y_array)

In [16]:
proj1 = read_img("F:\\SCI\\MOD\\DBAOD550\\tif\\MYD.DB.AOD550.200301.tif")[0]
proj1

'GEOGCS["unknown",DATUM["Unknown_based_on_Clarke_1866_ellipsoid",SPHEROID["Clarke 1866",6378206.4,294.978698213898]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

In [20]:
sr = osr.SpatialReference()
sr.SetProjCS( 'UTM 17 (WGS84) in northern hemisphere.' )
sr.SetWellKnownGeogCS( 'WGS84' )
sr.SetUTM( 17, True )
sr.IsGeographic()

0

In [21]:
proj.IsGeographic()

AttributeError: 'str' object has no attribute 'IsGeographic'